# Imports

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Introduction to the data

In [4]:
fp = 'C:\Users\Sam\Desktop\MPP\PROJECTS\DATAQUEST\PROJECTS\Guided Project- Preparing_data_for_SQLite\cademy_awards.csv'
academy_awards = pd.read_csv(fp,encoding='ISO-8859-1')
academy_awards.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10137 entries, 0 to 10136
Data columns (total 11 columns):
Year               10137 non-null object
Category           10137 non-null object
Nominee            10137 non-null object
Additional Info    9011 non-null object
Won?               10137 non-null object
Unnamed: 5         11 non-null object
Unnamed: 6         12 non-null object
Unnamed: 7         3 non-null object
Unnamed: 8         2 non-null object
Unnamed: 9         1 non-null object
Unnamed: 10        1 non-null object
dtypes: object(11)
memory usage: 871.2+ KB


In [5]:
academy_awards['Unnamed: 9'].value_counts()

*    1
Name: Unnamed: 9, dtype: int64

# Filtering the data

In [6]:
academy_awards[['Year']].head()

,Year
0,2010 (83rd)
1,2010 (83rd)
2,2010 (83rd)
3,2010 (83rd)
4,2010 (83rd)


In [7]:
academy_awards['Year'] = academy_awards['Year'].str[:4]
academy_awards['Year'] = academy_awards['Year'].astype(int)
academy_awards.head()

,Year,Category,Nominee,Additional Info,Won?,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,2010,Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},NO,NaN,NaN,NaN,NaN,NaN,NaN
1,2010,Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},NO,NaN,NaN,NaN,NaN,NaN,NaN
2,2010,Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},NO,NaN,NaN,NaN,NaN,NaN,NaN
3,2010,Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},YES,NaN,NaN,NaN,NaN,NaN,NaN
4,2010,Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},NO,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
later_than_2000 = academy_awards[academy_awards['Year'] > 2000]
award_categories = ['Actor -- Leading Role','Actor -- Supporting Role','Actress -- Leading Role',
                    'Actress -- Supporting Role']
nominations = later_than_2000[later_than_2000['Category'].isin(award_categories)]
nominations['Won?'].value_counts()

NO     160
YES     40
Name: Won?, dtype: int64

# Cleaning up the Won? and Unnamed columns

In [9]:
pd.options.mode.chained_assignment = None # default='warn'
replace_dict = {'YES': 1, 'NO': 0}
nominations['Won?'] = nominations['Won?'].map(replace_dict)
nominations['Won'] = nominations['Won?']
drop_cols = ['Won?','Unnamed: 5','Unnamed: 6','Unnamed: 7','Unnamed: 8',
             'Unnamed: 9', 'Unnamed: 10']
final_nominations = nominations.drop(drop_cols,axis=1)
final_nominations.head()

,Year,Category,Nominee,Additional Info,Won
0,2010,Actor -- Leading Role,Javier Bardem,Biutiful {'Uxbal'},0
1,2010,Actor -- Leading Role,Jeff Bridges,True Grit {'Rooster Cogburn'},0
2,2010,Actor -- Leading Role,Jesse Eisenberg,The Social Network {'Mark Zuckerberg'},0
3,2010,Actor -- Leading Role,Colin Firth,The King's Speech {'King George VI'},1
4,2010,Actor -- Leading Role,James Franco,127 Hours {'Aron Ralston'},0


# Cleaning up the Additional Info column

In [10]:
final_nominations['additional_info_one'] = final_nominations['Additional Info'].str.rstrip(" '}")
final_nominations['additional_info_two'] = final_nominations['additional_info_one'].str.split(" {'")
final_nominations['movie_names'] = final_nominations['additional_info_two'].str[0]
final_nominations['characters'] = final_nominations['additional_info_two'].str[1]
final_nominations['Movie'] = final_nominations['movie_names']
final_nominations['Character'] = final_nominations['characters']
final_nominations = final_nominations.drop(['Additional Info','additional_info_one','additional_info_two', 'movie_names',
                                            'characters'],axis=1)
final_nominations.head()

,Year,Category,Nominee,Won,Movie,Character
0,2010,Actor -- Leading Role,Javier Bardem,0,Biutiful,Uxbal
1,2010,Actor -- Leading Role,Jeff Bridges,0,True Grit,Rooster Cogburn
2,2010,Actor -- Leading Role,Jesse Eisenberg,0,The Social Network,Mark Zuckerberg
3,2010,Actor -- Leading Role,Colin Firth,1,The King's Speech,King George VI
4,2010,Actor -- Leading Role,James Franco,0,127 Hours,Aron Ralston


# Exporting to SQLite

In [15]:
import sqlite3
conn = sqlite3.connect('nominations.db')
final_nominations.to_sql("nominations",conn, index=False)

# Verifying in SQL

In [12]:
query_one = "select * from nominations LIMIT 10;"
query_two = "pragma table_info(nominations);"

results = conn.execute(query_one).fetchall()
results

[(2010, u'Actor -- Leading Role', u'Javier Bardem', 0, u'Biutiful', u'Uxbal'),
 (2010,
  u'Actor -- Leading Role',
  u'Jeff Bridges',
  0,
  u'True Grit',
  u'Rooster Cogburn'),
 (2010,
  u'Actor -- Leading Role',
  u'Jesse Eisenberg',
  0,
  u'The Social Network',
  u'Mark Zuckerberg'),
 (2010,
  u'Actor -- Leading Role',
  u'Colin Firth',
  1,
  u"The King's Speech",
  u'King George VI'),
 (2010,
  u'Actor -- Leading Role',
  u'James Franco',
  0,
  u'127 Hours',
  u'Aron Ralston'),
 (2010,
  u'Actor -- Supporting Role',
  u'Christian Bale',
  1,
  u'The Fighter',
  u'Dicky Eklund'),
 (2010,
  u'Actor -- Supporting Role',
  u'John Hawkes',
  0,
  u"Winter's Bone",
  u'Teardrop'),
 (2010,
  u'Actor -- Supporting Role',
  u'Jeremy Renner',
  0,
  u'The Town',
  u'James Coughlin'),
 (2010,
  u'Actor -- Supporting Role',
  u'Mark Ruffalo',
  0,
  u'The Kids Are All Right',
  u'Paul'),
 (2010,
  u'Actor -- Supporting Role',
  u'Geoffrey Rush',
  0,
  u"The King's Speech",
  u'Lionel Logue

In [13]:
conn.execute(query_two).fetchall()

[(0, u'Year', u'INTEGER', 0, None, 0),
 (1, u'Category', u'TEXT', 0, None, 0),
 (2, u'Nominee', u'TEXT', 0, None, 0),
 (3, u'Won', u'INTEGER', 0, None, 0),
 (4, u'Movie', u'TEXT', 0, None, 0),
 (5, u'Character', u'TEXT', 0, None, 0)]

In [ ]:
conn.close()